# This notebook lists the external & internal (linked services e.g.Lakehouse, Warehouse) connections
----------------------------------------------------------------------------------------------------------------
- **IdentifyExternalInternalConnections**
- **Version V1.0**

Please note that this notebook can used to identify & list external & internal connections. The values from this can be used in configuring the pipeline_deployment_config.yml file while performing automatic deployment of Microsoft Fabric data pipelines in a Fabric tenant.e.g. **Connection ID - f39ca659-7698-41f7-93c2-6ee3a115febc** need to be noted & can be used while updating externalreferences's oldconnectionid property value in pipeline_deployment_config.yml

- **PREREQUISITES**
- Please ensure that the **JSON files & YAML files** are kept at the lakehouse folder
----------------------------------------------------------------------------------------------------------------

In [ ]:
# import libraries
import json
import yaml

# Install jsonpath_ng library
- jsonpath_ng library provides functionality for querying JSON-like data structures using JSONPath expressions.

In [ ]:
%pip install jsonpath_ng

# Read and parse the configuration file

In [ ]:
base_folder_path = "/lakehouse/default/Files/" # modify based on your environment e.g. Files/ in Fabric
config_file_name = "identifyconnections_deployment_config.yml"
config_file_path = base_folder_path + config_file_name
config_data = None

# Read the config file
with open(config_file_path, 'r') as file:
    try:
        config_data = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

# Process the config data
pipeline_config_files = config_data.get("pipelinejsonfiles", [])

# Read JSON file from Lakehouse location

In [ ]:
def readJsonFile(file_path):
    try:
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            return json_data
            #print(json_data)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON in file '{file_path}'.")

# Find & print external connections in Json file

In [ ]:
from jsonpath_ng import parse

def find_and_print_connections(json_data):
    jsonpath_expr = parse('$..externalReferences.connection')
    
    # Find all matches
    matches = jsonpath_expr.find(json_data)
    
    # Initialize a count
    count = 0

    # Print all connection values
    for match in matches:
        external_references = match.value
        connection_value = match.value
        print(f"External Connection with Connection ID - {connection_value} found.")
        count += 1
    return count

# 

# Find & print internal connections/linkedservices like Datawarehouse & Lakehouse in Json file

In [ ]:
from jsonpath_ng import parse 
def find_and_print_linked_service_details(json_data):
    jsonpath_expr = parse('$..linkedService')
    matches = jsonpath_expr.find(json_data)
    
    # Initialize a count
    count = 0
    if matches:
        for match in matches:
            linked_service = match.value
        print(f"LinkedService - {linked_service['name']} of type {linked_service['properties']['type']} found.")
        count += 1
    return count

# List the external & internal (linked services e.g.Lakehouse, Warehouse) connections

In [ ]:
# Iterate over each item in pipeline_json_files
for item in pipeline_config_files:
    display_name = item.get("pipelinedisplayname")
    print(f"PROCESSING PIPELINE : {display_name}")
    file_path = item.get("pipelinejsonfilepath")
    json_file_value =  readJsonFile(file_path)
    extref_count = find_and_print_connections(json_file_value)
    print(f"Total {extref_count} externalReferences found for {display_name} pipeline.\n")
    intref_count = find_and_print_linked_service_details(json_file_value)
    print(f"Total {intref_count} LinkedServiceReferences found for {display_name} pipeline.")
    print("----------------------------------------------------------------------")
